In [1]:
import pandas as pd
import os

import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
# классификаторы
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
# метрики классификации
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, precision_recall_curve


# 1. Скачайте данные по ссылке
https://www.kaggle.com/competitions/titanic/data?select=train.csv 

In [2]:
# имя файла с данными
data_file_dir = u"./data"
data_file_names = [os.path.join(data_file_dir, x) for x in os.listdir(data_file_dir)]
data_file_names

['./data/test.csv', './data/train.csv', './data/gender_submission.csv']

In [3]:
# данные и их структура
print("==Данные и их структура из файла train")
data = pd.read_csv(data_file_names[1])
print(data.shape)
print(data.info())
data.sample(7)

==Данные и их структура из файла train
(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,PC 17754,34.6542,A5,C
557,558,0,1,"Robbins, Mr. Victor",male,NaN,0,0,PC 17757,227.5250,NaN,C
706,707,1,2,"Kelly, Mrs. Florence ""Fannie""",female,45.0,0,0,223596,13.5000,NaN,S
856,857,1,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",female,45.0,1,1,36928,164.8667,NaN,S
200,201,0,3,"Vande Walle, Mr. Nestor Cyriel",male,28.0,0,0,345770,9.5000,NaN,S
870,871,0,3,"Balkic, Mr. Cerin",male,26.0,0,0,349248,7.8958,NaN,S
616,617,0,3,"Danbom, Mr. Ernst Gilbert",male,34.0,1,1,347080,14.4000,NaN,S


In [4]:
# NaN мешают классификации - заменим их на 'none'
data.Cabin.unique()
data.Cabin.fillna('none', inplace=True)
data.Cabin.unique()

data.dropna(inplace=True)
data.shape

(712, 12)

In [5]:
# количество уникальных значений по каждому из признаков
print(data.shape)
data.nunique()

(712, 12)


PassengerId    712
Survived         2
Pclass           3
Name           712
Sex              2
Age             88
SibSp            6
Parch            7
Ticket         541
Fare           219
Cabin          134
Embarked         3
dtype: int64

In [6]:
# баланс целевого признака 
data['Survived'].value_counts(normalize=True)

0    0.595506
1    0.404494
Name: Survived, dtype: float64

# 2. Исключите признаки, которые на ваш взгляд, могут привести к переобучению 
(например, id).

In [7]:
print("==Исходный набор признаков:")
features = data.columns
list(features)

==Исходный набор признаков:


['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [8]:
print("==Набор признаков для обучения:")
# 
good_features = \
[\
 # 'PassengerId', # номер 
 # 'Survived', # Выживание 0 = Нет, 1 = Да
 'Pclass', # Класс билета 1 = 1-й, 2 = 2-й, 3 = 3-й
 # 'Name',  # имя - 
 'Sex',    # пол
 'Age',    # возраст в годах
 'SibSp',   # Количество братьев и сестер / супругов на борту "Титаника"
 'Parch',   # Количество родителей / детей на борту "Титаника"
 # 'Ticket', # номер билета
 'Fare',  # тариф
 'Cabin', # номер каюты
 'Embarked' # Порт посадки C = Шербур, Q = Квинстаун, S = Саутгемптон
]

good_features

==Набор признаков для обучения:


['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']

# 3. Разделите выборку на train и test.

In [9]:
# признаки для обучения
X = data[good_features]
# целевой признак 
y = data['Survived']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state=11)
print(X_train.shape)

(534, 8)


# 4. Преобразуйте признаки
 категориальные признаки с помощью sklearn.preprocessing.OneHotEncoder, 
 
 а некатегориальные с помощью sklearn.preprocessing.StandardScaler (на следующем занятии расскажем, зачем это нужно)

## Категориальные признаки

In [11]:
cat_features = \
[\
 # 'PassengerId', # номер 
 # 'Survived', # Выживание 0 = Нет, 1 = Да
 # 'Pclass', # Класс билета 1 = 1-й, 2 = 2-й, 3 = 3-й
 # 'Name',  # имя - 
 'Sex',    # пол
 # 'Age',    # возраст в годах
 # 'SibSp',   # Количество братьев и сестер / супругов на борту "Титаника"
 # 'Parch',   # Количество родителей / детей на борту "Титаника"
 # 'Ticket', # номер билета
 # 'Fare',  # тариф
 'Cabin', # номер каюты
 'Embarked' # Порт посадки C = Шербур, Q = Квинстаун, S = Саутгемптон
]
cat_features_scaled = [f+"_ohe" for f in cat_features]
cat_features_scaled

['Sex_ohe', 'Cabin_ohe', 'Embarked_ohe']

### Sex

In [12]:
# категориальные признаки с помощью sklearn.preprocessing.OneHotEncoder,
ohe = OneHotEncoder() #handle_unknown='error', drop='first' drop='if_binary'
sx=ohe.fit_transform(X_train['Sex'].values.reshape(-1,1) ).toarray()
columns = ['Sex_'+x for x in ohe.categories_[0]]
df_sex = pd.DataFrame(sx, columns=columns).iloc[:,0].to_frame() #, columns=ohe.categories_
df_sex
# X_train = pd.concat([X_train,df_sex], axis=1)
X_train_enc = df_sex
X_train_enc.sample(5)

# для тестового датасета
sx_t=ohe.fit_transform(X_test['Sex'].values.reshape(-1,1) ).toarray()
# columns = ['Sex_'+x for x in ohe.categories_[0]]
df_sex_t = pd.DataFrame(sx_t, columns=columns).iloc[:,0].to_frame() #, columns=ohe.categories_
# df_sex
# X_train = pd.concat([X_train,df_sex], axis=1)
X_test_enc = df_sex_t
# X_train_enc.sample(5)

### Embarked_

In [13]:
# категориальные признаки Embarked
ohe = OneHotEncoder() #handle_unknown='error', drop='first' drop='if_binary'
emb=ohe.fit_transform(X_train['Embarked'].values.reshape(-1,1) ).toarray()
columns=['Embarked_'+str(x) for x in list(ohe.categories_[0])]
df_emb = pd.DataFrame(emb, columns=columns)#.iloc[:,0].to_frame() #, columns=ohe.categories_
df_emb
X_train_enc = pd.concat([X_train_enc,df_emb], axis=1)
X_train_enc

# для тестового датасета
emb_t=ohe.fit_transform(X_test['Embarked'].values.reshape(-1,1) ).toarray()
# columns=['Embarked_'+str(x) for x in list(ohe.categories_[0])]
df_emb_t = pd.DataFrame(emb_t, columns=columns)#.iloc[:,0].to_frame() #, columns=ohe.categories_
# df_emb
X_test_enc = pd.concat([X_test_enc,df_emb_t], axis=1)
# X_test_enc

### Cabin

In [14]:
# # категориальные признаки Cabin - не будем его брать, тк в нем много пропусков 
# ohe = OneHotEncoder() #handle_unknown='error', drop='first' drop='if_binary'
# cab=ohe.fit_transform(X_train['Cabin'].values.reshape(-1,1) ).toarray()
# columns=['Cabin_'+str(x) for x in list(ohe.categories_[0])]
# df_cab = pd.DataFrame(cab, columns=columns)#.iloc[:,0].to_frame() #, columns=ohe.categories_
# df_cab
# X_train_enc = pd.concat([X_train_enc,df_cab], axis=1)
# X_train_enc


## Некатегориальные признаки

In [15]:
# удаляем пропущенные данные в поле Возраст
X_train.Age.unique()
# X_train.Age[X_train.Age == np.nan]
print(X_train.Age.dropna().shape)

# для тестового датасета
X_test.Age.unique()
# X_train.Age[X_train.Age == np.nan]
X_test.Age.dropna().shape

(534,)


(178,)

In [16]:
noncat_features = \
[\
 # 'PassengerId', # номер 
 # 'Survived', # Выживание 0 = Нет, 1 = Да
 'Pclass', # Класс билета 1 = 1-й, 2 = 2-й, 3 = 3-й
 # 'Name',  # имя - 
 # 'Sex',    # пол
 'Age',    # возраст в годах
 'SibSp',   # Количество братьев и сестер / супругов на борту "Титаника"
 'Parch',   # Количество родителей / детей на борту "Титаника"
 # 'Ticket', # номер билета
 'Fare',  # тариф
 # 'Cabin', # номер каюты
 # 'Embarked' # Порт посадки C = Шербур, Q = Квинстаун, S = Саутгемптон
]
noncat_features_scaled = [f+"_scaled" for f in noncat_features]
noncat_features_scaled

['Pclass_scaled', 'Age_scaled', 'SibSp_scaled', 'Parch_scaled', 'Fare_scaled']

In [17]:
scaler = StandardScaler()
X_train_enc[noncat_features_scaled] = scaler.fit_transform(X_train[noncat_features].values.reshape(-1,len(noncat_features)) )
X_train_enc

# для тестового датасета
X_test_enc[noncat_features_scaled] = scaler.fit_transform(X_test[noncat_features].values.reshape(-1,len(noncat_features)) )
X_test_enc


,Sex_female,Embarked_C,Embarked_Q,Embarked_S,Pclass_scaled,Age_scaled,SibSp_scaled,Parch_scaled,Fare_scaled
0,1.0,1.0,0.0,0.0,-1.404017,1.616298,0.681276,-0.532873,0.416903
1,0.0,0.0,0.0,1.0,0.964846,0.027723,-0.636845,-0.532873,-0.523717
2,1.0,1.0,0.0,0.0,-1.404017,0.061523,0.681276,-0.532873,1.403776
3,0.0,1.0,0.0,0.0,-1.404017,2.021892,0.681276,0.609915,0.779596
4,1.0,0.0,0.0,1.0,0.964846,-0.208873,-0.636845,0.609915,-0.442660
...,...,...,...,...,...,...,...,...,...
173,0.0,0.0,0.0,1.0,-0.219586,0.872710,0.681276,0.609915,-0.190332
174,0.0,0.0,0.0,1.0,-0.219586,-0.344071,0.681276,-0.532873,-0.194912
175,1.0,0.0,0.0,1.0,0.964846,-0.682066,-0.636845,-0.532873,-0.512497
176,1.0,0.0,0.0,1.0,-0.219586,0.940309,0.681276,-0.532873,-0.194912


# 5. Обучите модели логистической регрессии и KNN
, посчитайте все метрики классификации, которые были упомянуты на занятии, сделайте выводы.

In [18]:
knn = KNeighborsClassifier()
knn.fit(X_train_enc, y_train)

KNeighborsClassifier()

# тренировочный датасет

In [76]:
y_pred = knn.predict(X_train_enc)

In [74]:
print("Тренировочный датасет:")
print("confusion_matrix:\n", confusion_matrix(y_train, y_pred) )

print("recall_score:\n", recall_score(y_train, y_pred) )

print("precision_score:\n", precision_score(y_train, y_pred))

print("accuracy_score:\n", accuracy_score(y_train, y_pred) )

confusion_matrix:
 [[295  33]
 [ 51 155]]
recall_score:
 0.7524271844660194
precision_score:
 0.824468085106383
accuracy_score:
 0.8426966292134831


In [74]:
print("Тренировочный датасет:")
print("confusion_matrix:\n", confusion_matrix(y_train, y_pred) )

print("recall_score:\n", recall_score(y_train, y_pred) )

print("precision_score:\n", precision_score(y_train, y_pred))

print("accuracy_score:\n", accuracy_score(y_train, y_pred) )

confusion_matrix:
 [[295  33]
 [ 51 155]]
recall_score:
 0.7524271844660194
precision_score:
 0.824468085106383
accuracy_score:
 0.8426966292134831


# ТЕстовый датасет

In [21]:
y_pred_t = knn.predict(X_test_enc)

In [22]:
print("Тестовый датасет:")
print("confusion_matrix:\n", confusion_matrix(y_test, y_pred_t) )

print("recall_score:\n", recall_score(y_test, y_pred_t) )

print("precision_score:\n", precision_score(y_test, y_pred_t))

print("accuracy_score:\n", accuracy_score(y_test, y_pred_t) )

Тестовый датасет:
confusion_matrix:
 [[85 11]
 [23 59]]
recall_score:
 0.7195121951219512
precision_score:
 0.8428571428571429
accuracy_score:
 0.8089887640449438


# GIT